In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torchvision, torch

In [3]:
import sys, os
sys.path.insert(0,"../torchxrayvision/")
import torchxrayvision as xrv
import matplotlib.pyplot as plt
import torch
from torch.nn import functional as F
import glob
import numpy as np
import skimage, skimage.filters
%matplotlib inline
import captum, captum.attr
import torch, torch.nn
import pickle
import attribution
import pandas as pd
import datasets
import shutil
import json
import glob

In [4]:
def full_frame(width=None, height=None):
    import matplotlib as mpl
    mpl.rcParams['savefig.pad_inches'] = 0
    figsize = None if width is None else (width, height)
    fig = plt.figure(figsize=figsize)
    ax = plt.axes([0,0,1,1], frameon=False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.autoscale(tight=True)

In [5]:
ae = xrv.autoencoders.ResNetAE(weights="101-elastic").cuda()

In [6]:
models = [xrv.models.DenseNet(weights="all").cuda(),
#           xrv.models.DenseNet(weights="pc").cuda(),
#           xrv.models.DenseNet(weights="nih").cuda(),
#           xrv.models.DenseNet(weights="chex").cuda(),
          xrv.models.DenseNet(weights="mimic_ch").cuda(),
          xrv.baseline_models.jfhealthcare.DenseNet().cuda()]

In [7]:
model = models[0]
print(model.pathologies)
print(model)

['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'Lung Lesion', 'Fracture', 'Lung Opacity', 'Enlarged Cardiomediastinum']
XRV-DenseNet121-all


In [8]:
dmerge = datasets.get_data("nih-pc-rsna")

Merge_Dataset num_samples=173479
└ PC_Dataset num_samples=91658 views=['PA'] data_aug=None
└ RSNA_Pneumonia_Dataset num_samples=14511 views=['PA'] data_aug=None
└ NIH_Dataset num_samples=67310 views=['PA'] data_aug=None



In [9]:
for_eval = ['Cardiomegaly','Effusion',
          'Atelectasis', 'Consolidation', 'Mass',
          'Pneumothorax', 'Infiltration', 'Edema', 
          'Emphysema', 'Fibrosis',  'Pneumonia', 
          'Pleural_Thickening',
          'Hernia',
          'Lung Opacity']

In [10]:
# target = "Effusion"
# label = 1
# dwhere = np.where((dmerge.labels[:,dmerge.pathologies.index(target)]  == label))[0]
# dwhere[:10]

In [10]:
folder = "images3/"
temp_path = "/lscratch/joecohen/SDS-2342-ASDAA"

In [270]:
target = "Lung Opacity"
dwhere = np.where((dmerge.labels[:,dmerge.pathologies.index(target)]  == 1))[0]

idxs = []
ps = []
for idx in np.random.choice(dwhere,30):
    sample = dmerge[idx]
    image = torch.from_numpy(sample["img"]).unsqueeze(0).cuda()
    p = models[0](image)[:,models[0].pathologies.index(target)].detach().cpu()
    if p > 0.5:
        idxs.append(idx)
        ps.append(p)

In [271]:
np.array(idxs)[np.argsort(ps)]

array([102170, 101901, 100305, 100667, 102072,  93358,  92329, 100973,
        92254, 105514,  92359,  92962,  93206, 102564, 102564, 101097,
       102159,  97552,  97806, 103490, 101497, 103310,  94470,  92045])

In [272]:
np.array(ps)[np.argsort(ps)]

array([0.5403551 , 0.61227524, 0.64109164, 0.6490886 , 0.6506804 ,
       0.65122926, 0.68513167, 0.6968231 , 0.7174721 , 0.7226087 ,
       0.733233  , 0.74165726, 0.7482377 , 0.7734405 , 0.7734405 ,
       0.78462076, 0.7896426 , 0.8051019 , 0.80828714, 0.811821  ,
       0.82119316, 0.82872313, 0.8516433 , 0.9723157 ], dtype=float32)

In [274]:
# for idx in reversed(np.array(idxs)[np.argsort(ps)]):
#     print(idx)
#     sample = dmerge[idx]
#     image = torch.from_numpy(sample["img"]).unsqueeze(0).cuda()
#     r = attribution.generate_video(image, models[0], target, ae, temp_path, 
#                                            border=True, show=True)
#     display(r)

In [ ]:
# for t in for_eval:
#     print("to_plot[\"{}\"] = ".format(t))

In [11]:
to_plot = {}
to_plot["Cardiomegaly"] = 10201
to_plot["Effusion"] =  36121
to_plot["Atelectasis"] = 128243
to_plot["Consolidation"] = 68669
to_plot["Mass"] = 8865
to_plot["Pneumothorax"] = 145736
to_plot["Infiltration"] = 155917
to_plot["Edema"] = 130535
to_plot["Emphysema"] = 39046
to_plot["Fibrosis"] = 151247
to_plot["Pneumonia"] = 17137
to_plot["Pleural_Thickening"] = 20735
to_plot["Hernia"] = 126989
to_plot["Lung Opacity"] = 102564

In [59]:
target = "Emphysema"
sample = dmerge[to_plot[target]]
image = torch.from_numpy(sample["img"]).unsqueeze(0).cuda()
#attribution.generate_attributions(sample, models[0], target, ae, temp_path, dmerge)
attribution.generate_video(image, models[0], target, ae, temp_path, border=True, show=True)

[0.5127366] -1000 400
0
10
20
module load ffmpeg;ffmpeg -loglevel quiet -stats -y -i /lscratch/joecohen/SDS-2342-ASDAA/image-%d-a.png -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p 'videos/single-Emphysema_XRV-ResNetAE-101-elastic_XRV-DenseNet121-all_.mp4'


In [32]:
label = 1

for target in for_eval:
    print(target)

    idx = to_plot[target]

    metadata = {}
    sample = dmerge[idx]

    csv = dmerge.csv.iloc[idx]
    image = torch.from_numpy(sample["img"]).unsqueeze(0).cuda()

    try:
        uid = str(idx) + csv.patientid
    except:
        uid = str(idx)
    
    dataset_name = dmerge.datasets[sample["source"]].__class__.__name__
    fid = dataset_name + "-" + uid + "-" + target + "-" + str(label)
    basefname = folder + fid
    
    json_name = basefname + ".json"
    if (os.path.exists(json_name)):
        print("already computed")
        #continue

    metadata = {}
    metadata["id"] = fid
    metadata["idx"] = str(idx)
    metadata["dataset"] = dataset_name
    metadata["source"] = json.loads(csv.to_json())
    metadata["target"] = target
    metadata["label"] = label
    metadata["model_used"] = str(model)
    metadata["attribs"] = {}
    
    #######
    px = 1/plt.rcParams['figure.dpi']
    full_frame(image[0][0].shape[0]*px,image[0][0].shape[1]*px)

    plt.imshow(image[0][0].cpu().numpy(), interpolation='none', cmap="gray")

    rfname = basefname + '.png'
    print(rfname)
    metadata["attribs"]["image"] = rfname
    plt.savefig(rfname, bbox_inches='tight', pad_inches=0, transparent=False)
    plt.close()

    #######
    
    for model in models:

        if not target in model.pathologies:
            continue
            
        thisbasefname = basefname + "-" + str(model)
        
        metadata["attribs"]["latentshift-" + str(model)] = thisbasefname + "-latentshift.mp4"
        
#         if (os.path.exists(thisbasefname + "-latentshift.mp4")):
#             print("already computed")
#             continue

        print(thisbasefname)

        fname = attribution.generate_video(image, model, target, ae, temp_path, 
                                   target_filename=thisbasefname + "-latentshift", 
                                   border=False, show=False)
        
        ######
        px = 1/plt.rcParams['figure.dpi']
        full_frame(image[0][0].shape[0]*px,image[0][0].shape[1]*px)
        
        dimage = attribution.compute_attribution(image, "latentshift-max", model, target, ae=ae, threshold=True)
        plt.imshow(image.cpu().numpy()[0][0], interpolation='none', cmap="gray")
        dimage[dimage==0] = np.nan
        plt.imshow(dimage, interpolation='none', alpha=0.8, cmap="Reds");
        
        rfname = thisbasefname + "-latentshift-max" + '.png'
        print(rfname)
        metadata["attribs"]["latentshift-max-" + str(model)] = rfname
        plt.savefig(rfname, bbox_inches='tight', pad_inches=0, transparent=False)
        plt.close()
        ######
        
        
        

    with open(json_name, 'w') as outfile:
        json.dump(metadata, outfile)


Cardiomegaly
already computed
images3/PC_Dataset-10201199918783419519779794936126082408474409-Cardiomegaly-1.png
images3/PC_Dataset-10201199918783419519779794936126082408474409-Cardiomegaly-1-XRV-DenseNet121-all
[0.39048815] -70 1
0
10
20
module load ffmpeg;ffmpeg -loglevel quiet -stats -y -i /lscratch/joecohen/SDS-2342-ASDAA/image-%d-a.png -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p 'images3/PC_Dataset-10201199918783419519779794936126082408474409-Cardiomegaly-1-XRV-DenseNet121-all-latentshift.mp4'
[0.39048815] -70 1
images3/PC_Dataset-10201199918783419519779794936126082408474409-Cardiomegaly-1-XRV-DenseNet121-all-latentshift-max.png
images3/PC_Dataset-10201199918783419519779794936126082408474409-Cardiomegaly-1-XRV-DenseNet121-mimic_ch
[0.4034218] -60 1
0
10
20
module load ffmpeg;ffmpeg -loglevel quiet -stats -y -i /lscratch/joecohen/SDS-2342-ASDAA/image-%d-a.png -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p 'images3/PC_Dataset-10201199918783419519779794

[0.50143313] -1000 920
images3/NIH_Dataset-14573616568-Pneumothorax-1-XRV-DenseNet121-mimic_ch-latentshift-max.png
Infiltration
already computed
images3/NIH_Dataset-15591720779-Infiltration-1.png
images3/NIH_Dataset-15591720779-Infiltration-1-XRV-DenseNet121-all
[0.5184655] -750 220
0
10
20
module load ffmpeg;ffmpeg -loglevel quiet -stats -y -i /lscratch/joecohen/SDS-2342-ASDAA/image-%d-a.png -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p 'images3/NIH_Dataset-15591720779-Infiltration-1-XRV-DenseNet121-all-latentshift.mp4'
[0.5184655] -750 220
images3/NIH_Dataset-15591720779-Infiltration-1-XRV-DenseNet121-all-latentshift-max.png
Edema
already computed
images3/NIH_Dataset-13053510535-Edema-1.png
images3/NIH_Dataset-13053510535-Edema-1-XRV-DenseNet121-all
[0.0097415] -1000 550
0
10
20
module load ffmpeg;ffmpeg -loglevel quiet -stats -y -i /lscratch/joecohen/SDS-2342-ASDAA/image-%d-a.png -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p 'images3/NIH_Dataset-1305351

In [33]:
page = ""
page += """
<div style="width:100%;height:50px">
<img style="float:right;height:50px" src="assets/aimi-logo.png" />
<img style="float:right;height:50px" src="assets/stanford-logo2.png" />
</div>
<div class="main">
"""

page+= "<table><tr>\n"
page+= "<td>Image</td>\n"
for model in models:
    page += "<td>{}</td>\n".format(str(model))
page+= "</tr></table>\n"

for target in for_eval:
    
    print(target,str(model))
    j = glob.glob("images3/*{}*.json".format(target))[0]
    metadata = json.load(open(j))
    print(j)
    
    page += "<h2>Prediction of {}</h2>\n".format(target)
    page+= "<table><tr>\n"
    page += "<td><img src='{}'/></td>\n".format(metadata["attribs"]["image"])
        
    for model in models:
        if target in model.pathologies:
            page += "<td><img src='{}'/></td>\n".format(metadata["attribs"]["latentshift-max-" + str(model)])
            page += "<td><video autoplay loop muted><source id='latentshift' src='{}' type='video/mp4'>Your browser does not support the video tag.</video></td>\n".format(metadata["attribs"]["latentshift-" + str(model)])
        else:
            page += "<td>-</td>\n"
            page += "<td>-</td>\n"
    page+= "</tr></table>\n"
page+= "</div>\n"
with open("index3.htm", 'w') as f:
    f.write(page)

Cardiomegaly jfhealthcare-DenseNet121
images3/PC_Dataset-10201199918783419519779794936126082408474409-Cardiomegaly-1.json
Effusion jfhealthcare-DenseNet121
images3/PC_Dataset-36121320013082349055928370704331338105989249-Effusion-1.json
Atelectasis jfhealthcare-DenseNet121
images3/NIH_Dataset-1282439606-Atelectasis-1.json
Consolidation jfhealthcare-DenseNet121
images3/PC_Dataset-68669128681721738321741397205247540104929567-Consolidation-1.json
Mass jfhealthcare-DenseNet121
images3/PC_Dataset-8865137103320410885238934809914742203430307-Mass-1.json
Pneumothorax jfhealthcare-DenseNet121
images3/NIH_Dataset-14573616568-Pneumothorax-1.json
Infiltration jfhealthcare-DenseNet121
images3/NIH_Dataset-15591720779-Infiltration-1.json
Edema jfhealthcare-DenseNet121
images3/NIH_Dataset-13053510535-Edema-1.json
Emphysema jfhealthcare-DenseNet121
images3/PC_Dataset-39046269760008870352218360670210871191164426-Emphysema-1.json
Fibrosis jfhealthcare-DenseNet121
images3/NIH_Dataset-15124718782-Fibrosis-1